<a href="https://colab.research.google.com/github/gulsumaltintas/Natural_Language_Processing/blob/main/BagOfWords_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
import nltk
nltk.download('all')
import numpy as np
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [ ]:
import nltk
import re
import heapq

**Bag Of Words**

In [ ]:
par = """Every Saturday Daniel and his family go to the beach.
They live far from the beach, but once a week the family gets into the car and Daniel’s father drives for hours until they arrive.
Daniel’s parents love the beach. Daniel and his sister and brother love the beach.
The family’s dog loves the beach very much. But it is a problem to go to the beach every week.
Daniel’s father gets tired from driving so many hours.
The rest of the family gets tired from sitting in the car for so many hours.
Daniel’s mother says: “It’s fun in the beach, but it takes too much time to get there and back!” Daniel and his sister and brother are very sad.
They want to go to the beach, but it is a problem. They try to go the swimming pool, but it is not the same thing. One day Daniel’s parents come to talk with the kids.
They say: “We have a problem to go to the beach every week, but we love the beach, and you love the beach, and the dog loves the beach.
So we have a solution.We need to live near the beach!” Daniel and his sister and brother are very happy! Now they live near the beach.
They go to the beach every day!"""
#Metin Ön İşleme İşlemleri
#Sentence Tokenize
dataset = nltk.sent_tokenize(par)

#Lower the sentence
for i in range(len(dataset)):
  dataset[i] = dataset[i].lower()
  dataset[i] = re.sub(r'\W',' ',dataset[i]) #karakter olmayan her şeyi at,boşlukları at
  dataset[i] = re.sub(r'\s+',' ',dataset[i]) #birden fazla boşluk varsa onları tek boşluğa dönüştür

#Word Tokinezation bunun için histogram gerçekleştimemiz gerekiyor
#Histogram
wordcount = {}
for data in dataset:
  words = nltk.word_tokenize(data) #dataset içerisindeki her bir satırı word tokenize ediyoruz.Cümle içerisindeki sözcükler
  for word in words:
    if word not in wordcount.keys(): #Histogram oluşturmaya yarıyor.yoksa işlemi
      wordcount[word] = 1
    else: #varsa işlemi
      wordcount[word] +=1

#Sort the Histogram and Filter Words
freq = heapq.nlargest(100, wordcount, key= wordcount.get)#key değeri üzerinden sıralama yapacak 100 tanesi alacak
#En çok geçen ilk 100 sözcüğü aldık

#Creating Matrix
X = []

for data in dataset: #data her bir cümleye denk geliyor.BUnların liste içerisindeki her bir sözcük orada geçiyor mu geçmiyor mu?
  vector = []
  for word in freq:
    if word in nltk.word_tokenize(data):
      vector.append(1) # varsa 1 ekliyor
    else: #yoksa 0 ekliyor
      vector.append(0)
  X.append(vector)# Her bir cümlenin vektörünü ekleyeceğiz


X = np.asarray(X)
print(X)


[[1 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 0]
 [1 1 0 ... 0 0 1]
 [1 1 1 ... 0 0 0]]


**TF-IDF**

In [ ]:
#Metin Ön İşleme İşlemleri
#Sentence Tokenize
dataset = nltk.sent_tokenize(par)

#Lower the sentence
for i in range(len(dataset)):
  dataset[i] = dataset[i].lower()
  dataset[i] = re.sub(r'\W',' ',dataset[i]) #karakter olmayan her şeyi at,boşlukları at
  dataset[i] = re.sub(r'\s+',' ',dataset[i]) #birden fazla boşluk varsa onları tek boşluğa dönüştür

#Word Tokinezation bunun için histogram gerçekleştimemiz gerekiyor
#Histogram
wordcount = {}
for data in dataset:
  words = nltk.word_tokenize(data) #dataset içerisindeki her bir satırı word tokenize ediyoruz.Cümle içerisindeki sözcükler
  for word in words:
    if word not in wordcount.keys(): #Histogram oluşturmaya yarıyor.yoksa işlemi
      wordcount[word] = 1
    else: #varsa işlemi
      wordcount[word] +=1

#Sort the Histogram and Filter Words
freq = heapq.nlargest(100, wordcount, key= wordcount.get)#key değeri üzerinden sıralama yapacak 100 tanesi alacak
#En çok geçen ilk 100 sözcüğü aldık
#Buraya kadar bag of words ile aynı işlemler

#IDF Hesaplaması

word_idfs = {}

for word in freq:
  doc_count = 0
  for data in dataset:
    if word in nltk.word_tokenize(data):
      doc_count +=1 #dokümanlar içerisindeki geçiş sayısı
  word_idfs[word] = np.log(len(dataset)/(1+doc_count)) #1+ değerini 0 abölme işlemini ortadan kaldırmak için verdik

#TF Hesaplaması
tf_matrix = {}

for word in freq:
  doc_tf = []
  for data in dataset:
    fr = 0
    for w in nltk.word_tokenize(data):
      if word == w:
        fr +=1
    tf_word = fr / len(nltk.word_tokenize(data))
    doc_tf.append(tf_word)
  tf_matrix[word] = doc_tf

tfidf_matrix = []
for word in tf_matrix.keys():
  tfidf = []
  for value in tf_matrix[word]:
    score = value*word_idfs[word]
    tfidf.append(score)
  tfidf_matrix.append(tfidf)

X = np.asarray(tfidf_matrix)

X = np.transpose(X)#her bir satırın birer doküman olması için satır ve sütunların transpose unu aldık yani yerlerini değiştirdik
print(X)



[[0.         0.02076394 0.04700036 ... 0.         0.         0.        ]
 [0.         0.00798613 0.         ... 0.         0.         0.        ]
 [0.         0.03460656 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.00988759 0.02238113 ... 0.09902103 0.09902103 0.        ]
 [0.         0.03460656 0.         ... 0.         0.         0.34657359]
 [0.         0.02966277 0.06714338 ... 0.         0.         0.        ]]
